In [1]:
import os
import pandas as pd
import torch
from torch_geometric.data import Dataset, Data
import joblib
from flask import Flask, request
import ghhops_server as hs
import logging

In [2]:
device = torch.device('cpu')

In [3]:
app1 = Flask(__name__)
hops1 = hs.Hops(app1)
logging.basicConfig(level=logging.DEBUG)

In [5]:
# 加载保存的 MinMaxScaler
try:
    node_scaler = joblib.load('C_model/x_scaler.pkl')
    edge_scaler = joblib.load('C_model/edge_scaler.pkl')
    logging.info("node_scaler and edge_scaler loaded successfully.")
except Exception as e:
    logging.error(f"Error loading scaler: {e}")
    raise

[INFO] node_scaler and edge_scaler loaded successfully.


In [6]:
# 转化数据
class CustomDataset(Dataset):
    def __init__(self, node_file_path, edge_file_path, transform=None, pre_transform=None):
        super(CustomDataset, self).__init__(None, transform, pre_transform)
        self.node_file_path = node_file_path
        self.edge_file_path = edge_file_path

    @property
    def processed_file_names(self):
        return []

    def len(self):
        return 1  # 只有一个图数据

    def get(self, idx):
        try:
            nodes_df = pd.read_csv(self.node_file_path)
            edges_df = pd.read_csv(self.edge_file_path)

            # 处理节点数据
            nodes_df = nodes_df.drop(['Node_id', 'name', 'polygon'], axis=1, errors='ignore')
            nodes_df = nodes_df.apply(pd.to_numeric, errors='coerce').fillna(0)

            # 处理边数据
            edges_df = edges_df.apply(pd.to_numeric, errors='coerce').fillna(0)

            # 转换节点特征和边索引为Tensor
            node_features = torch.tensor(nodes_df.values, dtype=torch.float)
            edge_index = torch.tensor(edges_df[['ID1', 'ID2']].values, dtype=torch.long).t().contiguous()
            edge_attr = torch.tensor(edges_df[['length']].values, dtype=torch.float)

            # 创建Data对象，并添加graph_id属性
            graph_id = os.path.splitext(os.path.basename(self.node_file_path))[0]
            mydata = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, y=None)
            mydata.graph_id = graph_id  # 添加graph_id属性

            return mydata
        except Exception as e:
            logging.error(f"Error in data loading or processing: {e}")
            raise

def apply_normalization(features, scaler):
    try:
        normalized_features = scaler.transform(features)
        return torch.tensor(normalized_features, dtype=torch.float)
    except Exception as e:
        logging.error(f"Error in normalization: {e}")
        raise
        
# 执行标准化并保存
def convert_and_normalize(node_csv_path, edge_csv_path, save_folder):
    try:
        # 创建数据集和数据对象
        mydataset = CustomDataset(node_csv_path, edge_csv_path)
        mydata = mydataset[0]

        # 标准化图数据
        normalized_x = apply_normalization(mydata.x.numpy(), node_scaler)
        normalized_edge_attr = apply_normalization(mydata.edge_attr.numpy(), edge_scaler)

        # 创建标准化后的 Data 对象
        normalized_graph = Data(
            x=normalized_x,
            edge_index=mydata.edge_index,
            edge_attr=normalized_edge_attr,
            graph_id=mydata.graph_id
        )

        # 确保保存文件夹存在
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)

        # 使用与 node.csv 相同的文件名，并更改扩展名为 .pt
        node_filename = os.path.basename(node_csv_path)
        graph_filename = os.path.splitext(node_filename)[0] + ".pt"
        save_path = os.path.join(save_folder, graph_filename)

        # 保存标准化后的图数据
        torch.save(normalized_graph, save_path)
        logging.info(f"Normalized graph data saved to {save_path}")

        return save_path

    except Exception as e:
        logging.error(f"Error in graph conversion, normalization, or saving: {e}")
        raise        

In [ ]:
# 输入文件路径和保存文件夹
node_csv_path = "path_to_node.csv"
edge_csv_path = "path_to_edge.csv"
save_folder = "output_folder"

# 执行转换和标准化，并保存结果
graph_save_path = convert_and_normalize(node_csv_path, edge_csv_path, save_folder)
print(f"Graph saved at: {graph_save_path}")